In [5]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### 도구 (Tools)

도구(Tool) 에이전트, 체인 또는 LLM(Large Language Model)이 특정 작업을 수행하거나 외부 데이터에 접근할 수 있도록 돕는 기능을 제공합니다. <br>

Langchain에 통합된 도구를 사용할 수 있으며, 사용자 정의 도구를 사용할 수도 있습니다.<br>

[Langchain에 통합된 도구](https://python.langchain.com/v0.1/docs/integrations/tools/)

In [6]:
from langchain_teddynote import logging

logging.langsmith('PROJECT_2025_LANGCHAIN')

LangSmith 추적을 시작합니다.
[프로젝트명]
PROJECT_2025_LANGCHAIN


In [9]:
from langchain_experimental.tools import PythonAstREPLTool

# 파이썬 코드를 실행하는 도구 생성
python_tool = PythonAstREPLTool()   

In [10]:
answer = python_tool.invoke('print(100 + 200)')     # 코드를 실행 및 결과 리턴
print(answer)

300



###  LLM 에게 파이썬 코드를 작성하도록 요청하고 결과를 리턴

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

from langchain_experimental.tools import PythonAstREPLTool

In [12]:
def print_and_execute(code, debug=True):       # 파이썬 코드를 실행하고 중간 과정을 출력하고 실행결과를 리턴
    python_tool = PythonAstREPLTool()
    
    if debug:
        print('CODE:')
        print(code)

    return python_tool.invoke(code)

In [13]:
prompt = ChatPromptTemplate.from_messages(      # 파이썬 코드 작성을 지시하는 프롬프트
    [
        (
            'system',
            'You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. '
            'Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.',
        ),
        ('human', '{input}'),
    ]
)

In [ ]:
llm = ChatOpenAI( 
    api_key=key,
    model='gpt-4o-mini', 
    temperature=0
)

In [15]:
output_parser = StrOutputParser()

In [16]:
chain = prompt | llm | output_parser | RunnableLambda(print_and_execute)

In [19]:
print(chain.invoke('로또 번호 생성기를 출력하는 코드를 작성해주세요.', debug=True))

CODE:
import random

def generate_lotto_numbers():
    """Generate a set of 6 unique lotto numbers from 1 to 45."""
    return sorted(random.sample(range(1, 46), 6))

if __name__ == "__main__":
    print(generate_lotto_numbers())



### 검색 API 도구

TavilyAnswer 검색 API를 활용하여 검색 기능을 구현하는 도구. 
<br>

`TavilySearchResults` Tavily 검색 API를 실행하고 JSON 형식의 결과를 리턴.
<br>

매개변수: 

- max_results  (int) : 리턴 할 최대 검색 결과 개수 (default: 5)
- search_depth(str) : 검색 깊이(basic 또는 advanced) 
  - basic : 기본값
  - advanced : Tavily 검색 API가 더 깊고 광범위한 검색을 수행하도록 지시.
- include_domains (List[str]): 검색 결과에 포함할 도메인 목록
- exclude_domains (List[str]): 검색 결과에서 제외할 도메인 목록
- include_answer (bool): 원본 쿼리에 대한 짧은 답변을 포함 할 것인가
- include_raw_content (bool): 각 사이트의 정제된 HTML 콘텐츠를 포함할 것인지. 원본을 포함할 것인가
- include_images (bool): 쿼리 관련 이미지 목록 포함할 것인지

리턴 값:

- 검색 결과를 포함하는 JSON 문자열(uri, content)

In [20]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(                         # 도구 생성
    max_results=3,                                  # 검색 결과 수
    search_depth='advanced',                        # Tavily 검색 API가 더 깊고 광범위한 검색을 수행하도록 지시
    include_raw_content=True,                       # 검색 결과의 원본 내용 포함
    include_answer=True,                            # 검색 결과를 바탕으로 생성된 답변 포함
    include_images=True,                            # 검색 결과에 이미지 포함
    include_domains=['github.io', 'naver.com']      # 검색 대상 도메인을 github.io와 naver.com으로 제한
)

In [23]:
answer = tool.invoke({'query':'랭체인 Tool에 대해서 알려주세요'})

In [24]:
answer

[{'url': 'https://post.naver.com/viewer/postView.naver?volumeNo=38409843&vType=VERTICAL',
  'content': 'LangChain(랭체인) 은 대규모 언어 모델 (LLM) 을 활용한 애플리케이션 개발에 특화된 오픈소스 프레임워크입니다. LangChain 은 기존 언어 모델의 한계를 극복하고, AI 기술을 활용한 새로운 애플리케이션을 구축할 수 있는 중요한 도구로 자리 잡고 있습니다.'},
 {'url': 'https://blog.naver.com/PostView.naver?blogId=agapeuni&logNo=223590787053',
  'content': '[인공지능·AI] LLM 프레임워크 : 랭체인(LangChain)을 알아보자 : 네이버 블로그 NAVER 블로그 블로그 검색 블로그 주소 변경 불가 안내블로그 마켓 판매자의 이력 관리를 위해 블로그 주소 변경이 불가합니다. 블로그 아이디가 필요해요!블로그에서 진짜 나를 기록하고 블로그 아이디 만들기 블로그 아이디 만들기 레이어 닫기 설정한 아이디는 나중에 변경할 수 없으니 신중하게 입력해주세요. 변경 전 공유된 블로그/글/클립 링크는 연결이 끊길 수 있습니다. 블로그 도움말에서 아이디 변경 유의사항을 확인해보세요. 2. 블로그 아이디는 한번 변경하면 취소 확인 블로그 아이디는 한번 정하면 취소 확인 블로그 아이디가 만들어졌어요. 나중에 언제든지 변경할 수 있어요. 블로그 주제 주제 없음 블로그 시작하기 내 블로그  블로그 홈  프로필 사진 변경 알림 전체 알림 블로그 전체 서비스 보기 네이버 주요 서비스 바로가기 설정전체 서비스 보기 네이버 MYBOX 블로그 VIBE_New_ 블로그 메뉴 블로그  블로그'},
 {'url': 'https://m.blog.naver.com/ilovehandson/223124167133',
  'content': '\u200b\n스키마(4 항목)\n모델(3항목)\n프롬프트(4항목)\n인덱스(4항목)\

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(                         # 도구 생성
    max_results=3,                                  # 검색 결과 수
    search_depth='advanced',                        # Tavily 검색 API가 더 깊고 광범위한 검색을 수행하도록 지시
    include_raw_content=True,                       # 검색 결과의 원본 내용 포함
    include_answer=True,                            # 검색 결과를 바탕으로 생성된 답변 포함
    include_images=True,                            # 검색 결과에 이미지 포함
    include_domains=['google.com', 'naver.com']     # 검색 대상 도메인을 github.io와 naver.com으로 제한
)

answer = tool.invoke({'query':'인어공주에 대해서 알려주세요'})
print(answer)

[{'url': 'https://blog.naver.com/PostView.naver?blogId=justiiii&logNo=223748389874', 'content': '디즈니 애니메이션 인어공주 속에는 단순한 판타지가 아니라 역사적 배경과 깊은 교훈이 담겨 있어요. 대항해 시대의 탐험 정신, 산업혁명의 변화, 여성의 독립과 권리 등 다양한 역사적 요소를 자연스럽게 연결할 수 있답니다.'}, {'url': 'https://blog.naver.com/PostView.naver?blogId=h2you05&logNo=140007872446', 'content': '첫 댓글을 남겨보세요 공유하기'}, {'url': 'https://blog.naver.com/PostView.naver?blogId=dk9667&logNo=150145853611', 'content': '이 블로그에서 검색. 공감. 댓글 29'}]
